In [3]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [5]:
# run data prep file to be able to call dataframes
%run "mm_data_prep.ipynb"

Number of rows in denver is: 5388
<class 'pandas.core.frame.DataFrame'>
Index: 3844 entries, 0 to 5357
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           3844 non-null   int64  
 1   description                  3841 non-null   object 
 2   host_id                      3844 non-null   int64  
 3   host_since                   3844 non-null   object 
 4   host_has_profile_pic         3844 non-null   object 
 5   host_identity_verified       3844 non-null   object 
 6   neighbourhood_cleansed       3844 non-null   object 
 7   latitude                     3844 non-null   float64
 8   longitude                    3844 non-null   float64
 9   property_type                3844 non-null   object 
 10  room_type                    3844 non-null   object 
 11  accommodates                 3844 non-null   int64  
 12  bedrooms                     3844 non-null   fl

c:\Users\mered\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mered\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mered\AppData\Local\Temp\ipykernel_24160\2391358278.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = trans

In [4]:
# Check to see that I can access dataframe with outliers removed
denver_outliers_removed.head()

,id,description,host_id,host_since,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,amenities,price,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,city,bath_number,bath_text,listing_geometry,attraction1,attract1_geometry
0,360,Enjoy the famous Colorado weather and unplug i...,666,2008-07-08,t,t,Highland,39.766415,-105.002098,Entire guesthouse,Entire home/apt,3,2.0,2.0,"[""Extra pillows and blankets"", ""First aid kit""...",90.0,t,4,27,57,147,179,7,0,4.99,4.99,4.96,5.00,5.00,5.00,4.91,f,2.87,Denver,1.0,bath,POINT (-105.00210 39.76641),union_station,POINT (-105.00000 39.75306)
1,364,"Modern 1,000 square foot loft in the heart of ...",783,2008-07-11,t,t,Five Points,39.766720,-104.979060,Entire loft,Entire home/apt,3,1.0,1.0,"[""First aid kit"", ""Fire extinguisher"", ""Wifi"",...",179.0,t,23,53,83,358,87,0,0,4.85,4.78,4.81,4.95,4.96,4.65,4.71,f,0.50,Denver,1.5,baths,POINT (-104.97906 39.76672),union_station,POINT (-105.00000 39.75306)
7,31503,CORONA VIRUS RESPONSIBLE - ESSENTIAL WORKERS W...,135298,2010-05-30,t,t,West Highland,39.761790,-105.028450,Entire guest suite,Entire home/apt,2,1.0,1.0,"[""Radiant heating"", ""Extra pillows and blanket...",103.0,t,12,42,72,347,159,27,3,4.91,4.92,5.00,4.95,4.99,4.88,4.88,f,1.00,Denver,1.0,bath,POINT (-105.02845 39.76179),union_station,POINT (-105.00000 39.75306)
8,39405,Enjoy our oasis in the city and stay at one of...,666,2008-07-08,t,t,Highland,39.766053,-105.003078,Entire cottage,Entire home/apt,2,1.0,1.0,"[""Extra pillows and blankets"", ""Cooking basics...",136.0,t,6,7,32,114,667,35,3,4.92,4.88,4.93,4.98,4.97,4.85,4.85,f,4.17,Denver,1.0,bath,POINT (-105.00308 39.76605),union_station,POINT (-105.00000 39.75306)
10,154999,This condo is right in the heart of Downtown D...,745200,2011-06-26,t,t,CBD,39.744390,-104.989270,Entire condo,Entire home/apt,2,1.0,1.0,"[""Extra pillows and blankets"", ""Dishwasher"", ""...",162.0,t,0,0,0,139,11,0,0,4.70,4.40,4.70,4.70,4.90,4.90,4.60,f,0.07,Denver,1.0,bath,POINT (-104.98927 39.74439),union_station,POINT (-105.00000 39.75306)


In [5]:
id2 = denver_outliers_removed[['id']].reset_index(drop=True)
id2.head()


,id
0,360
1,364
2,31503
3,39405
4,154999


In [ ]:
###################### Transform property type ###########################################

In [21]:
# See percentages of property types to identify natural breaking point of most common property types
counts = denver_outliers_removed.value_counts('property_type')
counts_df = counts.to_frame('count')
total_count = counts_df['count'].sum()

counts_df['percent of total'] = ((counts_df['count'] / total_count) * 100).round(2)
counts_df.head(10)

,count,percent of total
property_type,,
Entire home,1219,34.19
Entire rental unit,663,18.60
Entire condo,414,11.61
Entire guest suite,408,11.44
Entire townhouse,375,10.52
Entire guesthouse,217,6.09
Entire bungalow,67,1.88
Entire loft,59,1.65
Room in boutique hotel,24,0.67


In [22]:
# Assign "other" property type to any property types not in top 5
top_property_types = denver_outliers_removed['property_type'].value_counts().nlargest(6).index
top_property_types

Index(['Entire home', 'Entire rental unit', 'Entire condo',
       'Entire guest suite', 'Entire townhouse', 'Entire guesthouse'],
      dtype='object', name='property_type')

In [26]:
# reduce property types to top 6 plus "other"

denver_outliers_removed.loc[:,'prop_type_reduced'] = denver_outliers_removed['property_type'].apply(lambda x: x if x in top_property_types else 'Other')
denver_outliers_removed.head()

,id,description,host_id,host_since,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,amenities,price,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,city,bath_number,bath_text,listing_geometry,attraction1,attract1_geometry,prop_type_reduced
0,360,Enjoy the famous Colorado weather and unplug i...,666,2008-07-08,t,t,Highland,39.766415,-105.002098,Entire guesthouse,Entire home/apt,3,2.0,2.0,"[""Extra pillows and blankets"", ""First aid kit""...",90.0,t,4,27,57,147,179,7,0,4.99,4.99,4.96,5.00,5.00,5.00,4.91,f,2.87,Denver,1.0,bath,POINT (-105.00210 39.76641),union_station,POINT (-105.00000 39.75306),Entire guesthouse
1,364,"Modern 1,000 square foot loft in the heart of ...",783,2008-07-11,t,t,Five Points,39.766720,-104.979060,Entire loft,Entire home/apt,3,1.0,1.0,"[""First aid kit"", ""Fire extinguisher"", ""Wifi"",...",179.0,t,23,53,83,358,87,0,0,4.85,4.78,4.81,4.95,4.96,4.65,4.71,f,0.50,Denver,1.5,baths,POINT (-104.97906 39.76672),union_station,POINT (-105.00000 39.75306),Other
7,31503,CORONA VIRUS RESPONSIBLE - ESSENTIAL WORKERS W...,135298,2010-05-30,t,t,West Highland,39.761790,-105.028450,Entire guest suite,Entire home/apt,2,1.0,1.0,"[""Radiant heating"", ""Extra pillows and blanket...",103.0,t,12,42,72,347,159,27,3,4.91,4.92,5.00,4.95,4.99,4.88,4.88,f,1.00,Denver,1.0,bath,POINT (-105.02845 39.76179),union_station,POINT (-105.00000 39.75306),Entire guest suite
8,39405,Enjoy our oasis in the city and stay at one of...,666,2008-07-08,t,t,Highland,39.766053,-105.003078,Entire cottage,Entire home/apt,2,1.0,1.0,"[""Extra pillows and blankets"", ""Cooking basics...",136.0,t,6,7,32,114,667,35,3,4.92,4.88,4.93,4.98,4.97,4.85,4.85,f,4.17,Denver,1.0,bath,POINT (-105.00308 39.76605),union_station,POINT (-105.00000 39.75306),Other
10,154999,This condo is right in the heart of Downtown D...,745200,2011-06-26,t,t,CBD,39.744390,-104.989270,Entire condo,Entire home/apt,2,1.0,1.0,"[""Extra pillows and blankets"", ""Dishwasher"", ""...",162.0,t,0,0,0,139,11,0,0,4.70,4.40,4.70,4.70,4.90,4.90,4.60,f,0.07,Denver,1.0,bath,POINT (-104.98927 39.74439),union_station,POINT (-105.00000 39.75306),Entire condo


In [27]:
# Create a list of columns with true/false data to be transformed using OneHotEncoder
# Also include 'prop_type_reduced' column
to_encode = ["host_has_profile_pic", "host_identity_verified", "has_availability", "instant_bookable", "prop_type_reduced"]

In [28]:
# Apply preprocessing steps selectively across different column types using ColumnTransformer from scikit-learn

preprocessor2 = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), filtered_col_names),
        ('bool', OneHotEncoder(drop='if_binary'), to_encode)])

denver_transformed_no_outliers = preprocessor2.fit_transform(denver_outliers_removed)

denver_transformed_no_outliers

array([[-0.56271452,  0.15265713, -0.19296682, ...,  0.        ,
         0.        ,  0.        ],
       [-0.56271452, -0.88593576, -0.93586826, ...,  0.        ,
         0.        ,  1.        ],
       [-0.99721253, -0.88593576, -0.93586826, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.74077948,  1.19125002,  0.54993461, ...,  0.        ,
         0.        ,  0.        ],
       [-0.99721253, -0.88593576, -0.93586826, ...,  0.        ,
         0.        ,  1.        ],
       [-0.99721253, -0.88593576, -0.93586826, ...,  0.        ,
         0.        ,  0.        ]])

In [29]:
# get transformed feature names

feature_names = preprocessor2.get_feature_names_out()

feature_names

array(['num__accommodates', 'num__bedrooms', 'num__beds', 'num__price',
       'num__availability_30', 'num__availability_60',
       'num__availability_90', 'num__availability_365',
       'num__number_of_reviews', 'num__number_of_reviews_ltm',
       'num__number_of_reviews_l30d', 'num__review_scores_rating',
       'num__review_scores_accuracy', 'num__review_scores_cleanliness',
       'num__review_scores_checkin', 'num__review_scores_communication',
       'num__review_scores_location', 'num__review_scores_value',
       'num__reviews_per_month', 'num__bath_number',
       'bool__host_has_profile_pic_t', 'bool__host_identity_verified_t',
       'bool__has_availability_t', 'bool__instant_bookable_t',
       'bool__prop_type_reduced_Entire condo',
       'bool__prop_type_reduced_Entire guest suite',
       'bool__prop_type_reduced_Entire guesthouse',
       'bool__prop_type_reduced_Entire home',
       'bool__prop_type_reduced_Entire rental unit',
       'bool__prop_type_reduced_Enti

In [30]:
# convert transformed array to DataFrame

transformed_no_outliers_df = pd.DataFrame(denver_transformed_no_outliers, columns=feature_names)


transformed_no_outliers_df.head()

,num__accommodates,num__bedrooms,num__beds,num__price,num__availability_30,num__availability_60,num__availability_90,num__availability_365,num__number_of_reviews,num__number_of_reviews_ltm,num__number_of_reviews_l30d,num__review_scores_rating,num__review_scores_accuracy,num__review_scores_cleanliness,num__review_scores_checkin,num__review_scores_communication,num__review_scores_location,num__review_scores_value,num__reviews_per_month,num__bath_number,bool__host_has_profile_pic_t,bool__host_identity_verified_t,bool__has_availability_t,bool__instant_bookable_t,bool__prop_type_reduced_Entire condo,bool__prop_type_reduced_Entire guest suite,bool__prop_type_reduced_Entire guesthouse,bool__prop_type_reduced_Entire home,bool__prop_type_reduced_Entire rental unit,bool__prop_type_reduced_Entire townhouse,bool__prop_type_reduced_Other
0,-0.562715,0.152657,-0.192967,-0.931810,-0.551344,0.120505,0.449145,-0.210263,1.146599,-0.460933,-0.714690,0.466510,0.414655,0.411728,0.344542,0.313652,0.550867,0.429767,0.369192,-0.637278,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.562715,-0.885936,-0.935868,0.207151,1.504547,1.454513,1.306215,1.467521,0.229000,-0.784757,-0.714690,0.013338,-0.286901,-0.104331,0.160479,0.184608,-0.641384,-0.122886,-0.750519,0.047441,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.997213,-0.885936,-0.935868,-0.765445,0.314295,0.890125,0.943609,1.380054,0.947121,0.464280,0.670775,0.207555,0.180803,0.549344,0.160479,0.281391,0.142096,0.346869,-0.514293,-0.637278,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.997213,-0.885936,-0.935868,-0.343133,-0.334934,-0.905654,-0.374960,-0.472665,6.013866,0.834366,0.670775,0.239924,0.047173,0.308516,0.270917,0.216869,0.039903,0.263971,0.983380,-0.637278,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.997213,-0.885936,-0.935868,-0.010403,-0.984163,-1.264810,-1.429815,-0.273876,-0.529017,-0.784757,-0.714690,-0.472203,-1.556384,-0.482775,-0.759839,-0.008959,0.210224,-0.426845,-0.953673,-0.637278,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# make sure id2 file and transformed_no_outliers has same row count

rows_id = len(id2)
rows_denv = len(transformed_no_outliers_df)
print(f"id2 rows {rows_id}")
print(f"rows denver {rows_denv}")

id2 rows 3565
rows denver 3565


In [32]:

transformed_no_outliers_df = pd.concat([id2, transformed_no_outliers_df], axis=1)
transformed_no_outliers_df.head()

,id,num__accommodates,num__bedrooms,num__beds,num__price,num__availability_30,num__availability_60,num__availability_90,num__availability_365,num__number_of_reviews,num__number_of_reviews_ltm,num__number_of_reviews_l30d,num__review_scores_rating,num__review_scores_accuracy,num__review_scores_cleanliness,num__review_scores_checkin,num__review_scores_communication,num__review_scores_location,num__review_scores_value,num__reviews_per_month,num__bath_number,bool__host_has_profile_pic_t,bool__host_identity_verified_t,bool__has_availability_t,bool__instant_bookable_t,bool__prop_type_reduced_Entire condo,bool__prop_type_reduced_Entire guest suite,bool__prop_type_reduced_Entire guesthouse,bool__prop_type_reduced_Entire home,bool__prop_type_reduced_Entire rental unit,bool__prop_type_reduced_Entire townhouse,bool__prop_type_reduced_Other
0,360,-0.562715,0.152657,-0.192967,-0.931810,-0.551344,0.120505,0.449145,-0.210263,1.146599,-0.460933,-0.714690,0.466510,0.414655,0.411728,0.344542,0.313652,0.550867,0.429767,0.369192,-0.637278,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,364,-0.562715,-0.885936,-0.935868,0.207151,1.504547,1.454513,1.306215,1.467521,0.229000,-0.784757,-0.714690,0.013338,-0.286901,-0.104331,0.160479,0.184608,-0.641384,-0.122886,-0.750519,0.047441,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,31503,-0.997213,-0.885936,-0.935868,-0.765445,0.314295,0.890125,0.943609,1.380054,0.947121,0.464280,0.670775,0.207555,0.180803,0.549344,0.160479,0.281391,0.142096,0.346869,-0.514293,-0.637278,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,39405,-0.997213,-0.885936,-0.935868,-0.343133,-0.334934,-0.905654,-0.374960,-0.472665,6.013866,0.834366,0.670775,0.239924,0.047173,0.308516,0.270917,0.216869,0.039903,0.263971,0.983380,-0.637278,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,154999,-0.997213,-0.885936,-0.935868,-0.010403,-0.984163,-1.264810,-1.429815,-0.273876,-0.529017,-0.784757,-0.714690,-0.472203,-1.556384,-0.482775,-0.759839,-0.008959,0.210224,-0.426845,-0.953673,-0.637278,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
null_counts_mm = transformed_no_outliers_df.isnull().sum().sort_values(ascending=False)
null_counts_over0_mm = null_counts_mm[null_counts_mm > 0]
print(null_counts_over0_mm)

Series([], dtype: int64)


In [34]:
# write to csv

transformed_no_outliers_df.to_csv("1_numeric_outliers_removed_transformed.csv", index=False)